In [2]:
from pdf2image import convert_from_path

pdf_path = '../QLoRA.pdf'

# convert PDF to images
images = convert_from_path(pdf_path)

# save each page as an image
for i, image in enumerate(images):
    image.save(f'page_{i + 1}.png', 'PNG')
    if i == 2:
        break

print("PDF has been converted to images successfully.")

PDF has been converted to images successfully.


In [9]:
from huggingface_hub import hf_hub_download
import re
from PIL import Image

from transformers import NougatProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch

processor = NougatProcessor.from_pretrained("facebook/nougat-base")
model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-base")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# prepare PDF image for the model
# filepath = hf_hub_download(repo_id="hf-internal-testing/fixtures_docvqa", filename="nougat_paper.png", repo_type="dataset")
filepath = "page_2.png"
image = Image.open(filepath)
pixel_values = processor(image, return_tensors="pt").pixel_values

# generate transcription (here we only generate 30 tokens)
outputs = model.generate(
    pixel_values.to(device),
    min_length=1,
    max_new_tokens=300,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
)

In [10]:
sequence = processor.batch_decode(outputs, skip_special_tokens=True)
sequence = processor.post_process_generation(sequence, fix_markdown=False)
# note: we're using repr here such for the sake of printing the \n characters, feel free to just print the sequence
print(repr(sequence))

['that are tuned by backpropagating gradients through the quantized weights.\n\nQLoRA reduces the average memory requirements of finetuning a 65B parameter model from \\(>\\)780GB of GPU memory to \\(<\\)48GB without degrading the runtime or predictive performance compared to a 16-bit fully finetuned baseline. This marks a significant shift in accessibility of LLM finetuning: now the largest publicly available models to date finetunable on a single GPU. Using QLoRA, we train the **Guanaco** family of models, with the second best model reaching 97.8% of the performance level of ChatGPT on the Vicuna [10] benchmark, while being trainable in less than 12 hours on a single consumer GPU; using a single professional GPU over 24 hours we achieve 99.3% with our largest model, essentially closing the gap to ChatGPT on the Vicuna benchmark. When deployed, our smallest **Guanaco** model (7B parameters) requires just 5 GB of memory and outperforms a 26 GB Alpaca model by more than 20 percentage po

In [12]:
markdown_file_path = 'QLoRA.md'

with open(markdown_file_path, 'w') as file:
    file.write(sequence[0])

print(f"Markdown content has been written to {markdown_file_path}")

Markdown content has been written to example.md
